# Scraping TranfertMarket


## Prerequisites

### Import required Libraries

In [1]:
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
import pandas as pd

### creating DataFrames

In [2]:
# crate empty DataFrames to store player data and contracts
PlayersTM = pd.DataFrame(columns=['Player', 'Age', 'Nation', 'Value'])
PlayersContracts = pd.DataFrame(columns=['Player', 'Contract'])

## Scraping phase

### Setting up league URLs

In [3]:
# defining TOP5 Leagues url variable
PLurl = "https://www.transfermarkt.fr/premier-league/startseite/wettbewerb/GB1"
Ligaurl = "https://www.transfermarkt.fr/laliga/startseite/wettbewerb/ES1"
Bundesligaurl = "https://www.transfermarkt.fr/bundesliga/startseite/wettbewerb/L1"
SerieAurl = "https://www.transfermarkt.fr/serie-a/startseite/wettbewerb/IT1"
Ligue1url = "https://www.transfermarkt.fr/ligue-1/startseite/wettbewerb/FR1"

#defining path variable for the ChromeDriver
path = "C:\\Users\\azedd\\Downloads\\chromedriver-win64\\chromedriver.exe"

### Let's create a function to scrape team links from league pages

In [ ]:
#create a function to colllecte teams website links

Links_list = []
def get_url(url):

    #set up the Chrome WebDriver with the specified path
    driver = webdriver.Chrome(service=webdriver.ChromeService(path))
    driver.get(url)

    # define a container to hold the teams table 
    container = driver.find_element(By.XPATH, '//*[@id="yw1"]/table/tbody')

    # search for team links within the container
    team_links = container.find_elements(By.TAG_NAME, 'a')
    team_urls = [link.get_attribute('href') for link in team_links]

    # adding the team URLs to the Links_list if they are not already present
    for link in team_urls:
        if link not in Links_list:
            Links_list.append(link)
    
    # close the driver
    driver.quit()
    return None

### Now let's apply our function to collect team links from the top 5 European leagues

In [5]:
get_url(PLurl)
get_url(Ligaurl)    
get_url(Bundesligaurl)
get_url(SerieAurl)
get_url(Ligue1url)
    

### Remove Duplicates

In [ ]:
# Remove the element at index 1
Links_list.pop(1)
Links_list.pop(41)
Links_list.pop(83)
Links_list.pop(118)
Links_list.pop(157)               
Links = []
# Remove duplicates
for i in range(len(Links_list)):
    if i % 2 == 0 and Links_list[i] not in Links:
        Links.append(Links_list[i])

In [10]:
len(Links)

94

### Scraping function

In [57]:
# create a function to collect player data and contracts
def collect_player_data(url):

    # Initialize the Chrome driver
    driver = webdriver.Chrome(service=webdriver.ChromeService(path))
    driver.get(url)

    #search for the table containing player data
    ind = 1
    
    while True:
        # search for the row containing player data
        row = driver.find_element(By.XPATH, f'//*[@id="yw1"]/table/tbody/tr[{ind}]')
        print(row.text)

        try:
            print(ind)
            player_name = row.find_element(By.XPATH, f'//*[@id="yw1"]/table/tbody/tr[{ind}]/td[2]/table/tbody/tr[1]/td[2]/a').text
            print(player_name)
            age = row.find_element(By.XPATH, f'//*[@id="yw1"]/table/tbody/tr[{ind}]/td[3]').text
            print(age)
            nation = row.find_element(By.XPATH, f'//*[@id="yw1"]/table/tbody/tr[{ind}]/td[4]/img').get_attribute('title')
            print(nation)
            value = row.find_element(By.XPATH, f'//*[@id="yw1"]/table/tbody/tr[{ind}]/td[5]/a').text
            print(value)
            PlayersTM.loc[len(PlayersTM)] = [player_name, age, nation, value]
            ind += 1
            # Go back to the team page
        except NoSuchElementException :
            print("Well done, no more players in this team.")
            break
        except ElementClickInterceptedException:
            print("Element click intercepted, skipping row.")
            ind += 1
        

    driver.quit()
    return None
    



In [58]:

collect_player_data("https://www.transfermarkt.fr/montpellier-hsc/startseite/verein/969/saison_id/2024")

40
Benjamin Lecomte 
Gardien de but
26 avr. 1991 (34) 1,50 mio. €  
1
Benjamin Lecomte 
26 avr. 1991 (34)
France
1,50 mio. €
16
Dimitry Bertaud
Gardien de but
6 juin 1998 (27)
1,00 mio. €  
2
Dimitry Bertaud
6 juin 1998 (27)
RD Congo
1,00 mio. €
1
Belmin Dizdarevic
Gardien de but
9 août 2001 (23) 100 K €  
3


Bosnie-Herzégovine


4


Suisse


5


Côte d'Ivoire


6


Mali


7


Mali


8


Guadeloupe


9


France


10


Serbie


11


Guinée


12


France


13


France


14


Cameroun


15


Mali


16


France


17


France


18


Maroc


19


France


20


France


21


France


22


Tunisie


23


France


24


Maroc


25


Algérie


26


Émirats Arabes Unis


27


Cameroun
Well done, no more players in this team.


In [ ]:
PlayersTM 

,Player,Age,Nation,Value
0,Benjamin Lecomte,26 avr. 1991 (34),France,"1,50 mio. €"
1,Dimitry Bertaud,6 juin 1998 (27),RD Congo,"1,00 mio. €"
2,,,Bosnie-Herzégovine,
3,,,Suisse,
4,,,Côte d'Ivoire,
5,,,Mali,
6,,,Mali,
7,,,Guadeloupe,
8,,,France,
9,,,Serbie,
